In [22]:
import pandas as pd
import math
file_path = 'dose_rsponse.xlsx'
df = pd.read_excel(file_path)
df = df[["CELL_LINE_NAME","TCGA_DESC","DRUG_NAME", "LN_IC50", "AUC", "SANGER_MODEL_ID"]]
df.columns = ["Cell line", "TCGA classification", "DRUG_NAME", "IC50", "AUC", "model_id"]
df["IC50"] = df["IC50"].apply(lambda x: math.exp(x))
df

,Cell line,TCGA classification,DRUG_NAME,IC50,AUC,model_id
0,ES5,UNCLASSIFIED,Erlotinib,52.815938,0.985678,SIDM00263
1,ES7,UNCLASSIFIED,Erlotinib,14.762497,0.972690,SIDM00269
2,EW-11,UNCLASSIFIED,Erlotinib,11.917287,0.944459,SIDM00203
3,SK-ES-1,UNCLASSIFIED,Erlotinib,7.641271,0.950758,SIDM01111
4,COLO-829,SKCM,Erlotinib,19.414244,0.954778,SIDM00909
...,...,...,...,...,...,...
333156,SNU-1040,COREAD,I-CBP112,161.627451,0.972251,SIDM00217
333157,SNU-61,COREAD,I-CBP112,306.555557,0.976109,SIDM00194
333158,SNU-81,COREAD,I-CBP112,138.483751,0.970851,SIDM00193
333159,SNU-C5,COREAD,I-CBP112,94.801383,0.972330,SIDM00498


In [12]:
td = pd.DataFrame(df['DRUG_NAME'].unique())
td.columns = ["DRUG_NAME"]
td

,DRUG_NAME
0,Erlotinib
1,Rapamycin
2,Sunitinib
3,PHA-665752
4,MG-132
...,...
373,TTK_3146
374,SN-38
375,Pevonedistat
376,PFI-3


In [23]:
def getdrug(drug):
    res  = df2[ df2["DRUG_NAME"] == drug]
    res = res.drop('DRUG_NAME', axis=1)
    if res.empty:
        return
    res.to_csv("./data/"+drug.replace("/","").replace(" ","")+".csv", index=False)

In [24]:
drug = pd.read_csv("sup.csv")
for d in list(drug["Drug"]):
    getdrug(d)

In [18]:
import pandas as pd
import numpy as np
drug = pd.read_csv("sup.csv")
drug["P"] = drug["P"].apply(lambda x: -np.log(x))
data = list(drug.itertuples(index=False, name=None))
data = sorted(data, key=lambda x: -abs(x[1]))
data = data[:20]
data = [t[0] for t in data]
print(data)
drug = drug.sort_values(by=['P'], ascending=False)
drug

['Dasatinib', 'Vorinostat', 'TL-2-105', 'Docetaxel', 'Bleomycin (50 uM)', '17-AAG', 'AC220', 'Trametinib', 'KIN001-102', 'I-BET', 'Tubastatin A', 'ABT-263', 'MP470', 'RDEA119 (rescreen)', 'RDEA119', 'KIN001-260', 'WZ3105', 'NPK76-II-72-1', 'UNC0638', 'AR-42']


,Drug,ρ,P
1,TL-2-105,-0.39,75.700129
0,Vorinostat,-0.39,70.016601
195,Bleomycin (50 uM),0.37,68.878702
2,AC220,-0.36,66.688790
197,Docetaxel,0.38,66.165202
...,...,...,...
238,CGP-082996,0.00,0.062940
237,Z-LLNle-CHO,0.00,0.056570
236,SB 216763,0.00,0.055513
234,BX-795,0.00,0.016129


In [20]:
import pandas as pd
import numpy as np
import math
def depth2(data):
    #input is a pandas data frame with gene expression values
    dt = data.T
    result = dt.apply(lambda col: (col-col.mean())/col.std())
    result = result.replace(np.nan,0)
    result = result.apply(lambda col: abs(col)).T
    return result.std()

In [58]:
def normalize(df):
    return df.applymap(lambda x: np.log2(x+1))

In [108]:
import json
import pandas as pd

def TCGA_GeneExpression(sample_sheet,gdcfolder):
    #takes in metacart & gdc folder containing after downloading cart from TCGA website
    #returns dataframe of gene expression values and patient ids
    f = pd.read_csv(sample_sheet, sep='\t')
    f = f.loc[f['Sample Type'] == "Primary Tumor"]
    df = None
    for patient in zip(f['File ID'], f['File Name'], f["Sample ID"]):
        gene_exp_file = patient[0]+"/"+patient[1]
        patient_id = patient[2]
        dt = pd.read_csv(gdcfolder+gene_exp_file, sep='\t',skiprows=[0])
        dt = dt[(dt['gene_type'] == 'protein_coding')] 
        if df is None:
            df = dt[["gene_name"]]
        dt = dt[["tpm_unstranded"]]
        dt.columns = [patient_id]
        if patient_id in df.columns:
            continue
        df = df.join(dt)
    df = df.set_index("gene_name")
    return df

In [90]:
df

,TCGA-BP-4781-01A,TCGA-CJ-4644-01A,TCGA-CZ-5986-01A,TCGA-BP-5176-01A,TCGA-CZ-4863-01A,TCGA-B8-A54D-01A,TCGA-A3-3387-01A,TCGA-A3-3346-01A,TCGA-CZ-5982-01A,TCGA-A3-3306-01A,...,TCGA-T7-A92I-01A,TCGA-AK-3458-01A,TCGA-B0-4703-01A,TCGA-B0-5106-01A,TCGA-B0-4846-01A,TCGA-CJ-4643-01A,TCGA-BP-5007-01A,TCGA-CJ-4901-01A,TCGA-BP-4349-01A,TCGA-BP-4982-01A
gene_name,,,,,,,,,,,,,,,,,,,,,
TSPAN6,15.8636,9.9606,11.3048,14.7617,10.4692,11.9531,7.8720,10.1982,6.9747,16.8472,...,10.4505,14.5275,5.7254,9.0256,15.5576,10.1907,10.6271,14.6265,10.1267,9.0345
TNMD,0.0611,0.2913,0.0730,1.3034,0.1584,0.1445,0.1825,0.0121,0.0236,0.0519,...,0.0104,0.3798,0.0000,0.1168,0.1414,0.6688,0.1711,0.0120,1.0498,0.1336
DPM1,21.2362,21.8412,23.0171,12.9768,24.5723,23.3969,21.4689,31.8542,21.7876,30.5257,...,20.0003,34.4250,23.8077,24.6971,27.6842,21.9829,19.0330,23.4316,20.9402,17.4117
SCYL3,1.8384,1.5417,2.0240,1.0763,1.9971,2.0142,1.7799,1.9794,1.8244,1.5495,...,1.9984,1.0834,2.0115,1.8231,2.0799,2.2674,1.6839,1.2976,1.7549,1.8961
C1orf112,0.7216,0.4026,0.7673,0.3030,0.6944,0.8289,0.8893,0.7647,0.6468,0.6667,...,0.5592,0.5715,1.0126,0.6739,0.5769,0.7954,0.5949,0.6530,0.5777,0.8410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AL451106.1,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
AC008763.4,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0044,0.0000,...,0.0000,0.0061,0.0044,0.0000,0.0000,0.0000,0.0037,0.0044,0.0000,0.0000
AC006486.3,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [89]:
df = TCGA_GeneExpression("./cancer_data/kirc/KIRC_Sample.tsv","./cancer_data/kirc/")
kirc = normalize(df)
kirc

,TCGA-BP-4781-01A,TCGA-CJ-4644-01A,TCGA-CZ-5986-01A,TCGA-BP-5176-01A,TCGA-CZ-4863-01A,TCGA-B8-A54D-01A,TCGA-A3-3387-01A,TCGA-A3-3346-01A,TCGA-CZ-5982-01A,TCGA-A3-3306-01A,...,TCGA-T7-A92I-01A,TCGA-AK-3458-01A,TCGA-B0-4703-01A,TCGA-B0-5106-01A,TCGA-B0-4846-01A,TCGA-CJ-4643-01A,TCGA-BP-5007-01A,TCGA-CJ-4901-01A,TCGA-BP-4349-01A,TCGA-BP-4982-01A
gene_name,,,,,,,,,,,,,,,,,,,,,
TSPAN6,4.075841,3.454255,3.621149,3.978351,3.519693,3.695226,3.149259,3.485195,2.995430,4.157626,...,3.517339,3.956754,2.749620,3.325617,4.049422,3.484228,3.539419,3.965923,3.475954,3.326897
TNMD,0.085561,0.368824,0.101650,1.203765,0.212134,0.194717,0.241840,0.017352,0.033652,0.072998,...,0.014927,0.464459,0.000000,0.159371,0.190804,0.738811,0.227864,0.017209,1.035483,0.180912
DPM1,4.474838,4.513567,4.585990,3.804962,4.676510,4.608626,4.489858,5.038006,4.510177,4.978456,...,4.392338,5.146696,4.632716,4.683534,4.842184,4.522489,4.324307,4.610676,4.455505,4.202551
SCYL3,1.505078,1.345794,1.596458,1.054015,1.583567,1.591775,1.475033,1.575022,1.497944,1.350214,...,1.584193,1.058940,1.590482,1.497280,1.622884,1.708143,1.424331,1.200128,1.462000,1.534111
C1orf112,0.783750,0.488104,0.821547,0.381837,0.760774,0.870976,0.917852,0.819423,0.719665,0.736994,...,0.640806,0.652142,1.009060,0.743213,0.657091,0.844305,0.673466,0.725087,0.657823,0.880490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AL451106.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
AC008763.4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006334,0.000000,...,0.000000,0.008774,0.006334,0.000000,0.000000,0.000000,0.005328,0.006334,0.000000,0.000000
AC006486.3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [91]:
def generate_subset(gene_expression, file, outputname):
    ith_values = pd.read_csv(file)
    patient_ids = list(ith_values.columns)
    subset_expression = gene_expression[patient_ids]
    res = depth2(subset_expression).to_frame().T
    res.to_csv(outputname, index = False)
#generate_subset(df, outputname, outputname)

In [92]:
import os
directory = "outputs_2"
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    cancer = file[:file.find("_")]
    new_name = file[:file.find(".")]+"_depth2.csv"
    if cancer=="kirc" and "unfiltered" not in filename:
        print(filename)
        generate_subset(kirc, "./"+directory+"/"+filename, "./outputs_3/"+new_name)


kirc_supervised_test_fold_1_200_genes_filtered.csv
kirc_unsupervised_train_fold_2_200_genes_filtered.csv
kirc_supervised_train_fold_2_100_genes_filtered.csv
kirc_unsupervised_train_fold_2_100_genes_filtered.csv
kirc_supervised_train_fold_1_200_genes_filtered.csv
kirc_unsupervised_test_fold_1_200_genes_filtered.csv
kirc_unsupervised_train_fold_0_200_genes_filtered.csv
kirc_supervised_test_fold_0_100_genes_filtered.csv
kirc_unsupervised_test_fold_2_100_genes_filtered.csv
kirc_supervised_test_fold_2_100_genes_filtered.csv
kirc_supervised_test_fold_1_100_genes_filtered.csv
kirc_unsupervised_test_fold_2_200_genes_filtered.csv
kirc_unsupervised_train_fold_1_200_genes_filtered.csv
kirc_supervised_train_fold_0_200_genes_filtered.csv
kirc_supervised_test_fold_0_200_genes_filtered.csv
kirc_unsupervised_train_fold_0_100_genes_filtered.csv
kirc_unsupervised_test_fold_1_100_genes_filtered.csv
kirc_unsupervised_train_fold_1_100_genes_filtered.csv
kirc_supervised_train_fold_0_100_genes_filtered.csv
k

In [95]:
import os
directory = "outputs_2"
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if "test" in filename:
        continue
    f1 = filename 
    f2 = filename.replace("train", "test")
    d1 = pd.read_csv("./"+directory+"/"+f1)
    d2 = pd.read_csv("./"+directory+"/"+f2)
    d1 = d1.join(d2)
    d1.to_csv("./full/"+filename.replace("train",""), index=False)
d1

,TCGA-OR-A5K0-01A,TCGA-OR-A5L6-01A,TCGA-OR-A5LT-01A,TCGA-OR-A5JZ-01A,TCGA-OR-A5JK-01A,TCGA-OR-A5JT-01A,TCGA-OR-A5LE-01A,TCGA-OR-A5J9-01A,TCGA-OR-A5LK-01A,TCGA-OR-A5L3-01A,...,TCGA-OR-A5JW-01A,TCGA-P6-A5OG-01A,TCGA-OR-A5J8-01A,TCGA-OR-A5JA-01A,TCGA-OR-A5K1-01A,TCGA-OR-A5K3-01A,TCGA-OR-A5J5-01A,TCGA-OR-A5KO-01A,TCGA-OR-A5JV-01A,TCGA-OR-A5KW-01A
0,0.403657,0.359599,0.407906,0.603898,0.245436,0.227015,0.827102,0.292267,0.330011,0.387883,...,0.329613,1.132296,1.032628,0.526283,0.290121,0.341483,0.571205,0.657813,0.559314,0.30785


In [109]:
df = TCGA_GeneExpression("./cancer_data/brca/BRCA_Samplet.tsv","./cancer_data/brca/")
brca = normalize(df)
brca

,TCGA-B6-A0RH-01A,TCGA-BH-A1FU-01A,TCGA-AR-A0TX-01A,TCGA-A1-A0SE-01A,TCGA-OL-A5D6-01A,TCGA-E2-A1IK-01A,TCGA-AN-A0FT-01A,TCGA-A2-A3KD-01A,TCGA-A2-A0EO-01A,TCGA-A2-A04P-01A,...,TCGA-E9-A1N6-01A,TCGA-AN-A0G0-01A,TCGA-E9-A3X8-01A,TCGA-BH-A0H0-01A,TCGA-E2-A10C-01A,TCGA-A8-A094-01A,TCGA-AC-A5XU-01A,TCGA-AR-A0U2-01A,TCGA-BH-A1EW-01A,TCGA-EW-A6SC-01A
gene_name,,,,,,,,,,,,,,,,,,,,,
TSPAN6,4.372144,6.076595,5.251742,6.164903,4.487338,4.506926,4.538302,5.515955,5.631229,5.102100,...,3.700928,6.152970,4.641887,5.520576,5.971672,5.896708,3.438652,3.473995,5.520171,4.786528
TNMD,0.000000,0.214871,0.821139,1.018207,0.128293,0.356933,0.608242,0.831229,0.802483,0.179129,...,0.312084,0.051998,1.181802,1.990084,0.571046,0.000000,0.186120,0.302056,1.253808,1.118626
DPM1,6.794521,6.362917,8.801638,6.369144,6.037330,6.405075,7.139578,6.125847,6.469471,4.864746,...,6.903614,6.243642,5.695086,6.548081,6.672396,7.693733,6.481375,7.489155,7.264114,6.575231
SCYL3,3.701815,3.852259,4.779139,4.104966,3.323327,2.695326,2.887369,4.117645,3.991690,2.226693,...,3.585636,4.633768,2.554392,4.645137,3.595730,4.055491,3.642320,4.496948,4.671378,3.593868
C1orf112,2.911078,3.114900,2.652739,3.889133,1.878647,1.604119,1.107018,2.957952,2.057450,1.422448,...,3.047870,4.280058,1.205955,4.210911,2.392867,3.389347,2.287206,4.466842,3.698496,2.874522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AL451106.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
AC008763.4,0.000000,0.000000,0.000000,0.000000,0.024603,0.000000,0.000000,0.019631,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.021480,0.000000,0.000000,0.000000,0.000000,0.000000
AC006486.3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [112]:
res = depth2(brca).to_frame().T
res

,TCGA-B6-A0RH-01A,TCGA-BH-A1FU-01A,TCGA-AR-A0TX-01A,TCGA-A1-A0SE-01A,TCGA-OL-A5D6-01A,TCGA-E2-A1IK-01A,TCGA-AN-A0FT-01A,TCGA-A2-A3KD-01A,TCGA-A2-A0EO-01A,TCGA-A2-A04P-01A,...,TCGA-E9-A1N6-01A,TCGA-AN-A0G0-01A,TCGA-E9-A3X8-01A,TCGA-BH-A0H0-01A,TCGA-E2-A10C-01A,TCGA-A8-A094-01A,TCGA-AC-A5XU-01A,TCGA-AR-A0U2-01A,TCGA-BH-A1EW-01A,TCGA-EW-A6SC-01A
0,0.5611,0.511886,0.550671,0.400706,0.694482,0.588508,0.802273,0.485406,0.398818,0.884874,...,0.514987,0.776278,0.856103,0.609304,0.635273,0.661661,0.664629,0.933727,0.563718,0.494145


In [113]:
res.to_csv("./brca_full.csv", index=False)

In [18]:
import pandas as pd
df = pd.read_csv("./r.csv", index_col=0)
df = df.sort_values(by=['Pval'])
#df = df['Pval'].apply(lambda x: round(x, 3))
df

,Name,Pval
11,acc_unsupervised__fold_2_100_genes_filtered,5.763423e-07
28,kirc_supervised__fold_1_200_genes_filtered,5.882703e-06
9,acc_unsupervised__fold_1_100_genes_filtered,1.139464e-04
12,acc_unsupervised__fold_2_200_genes_filtered,1.579201e-04
27,kirc_supervised__fold_1_100_genes_filtered,1.743035e-04
26,kirc_supervised__fold_0_200_genes_filtered,3.754536e-03
4,acc_supervised__fold_1_200_genes_filtered,4.103446e-03
22,brca_unsupervised__fold_1_200_genes_filtered,1.685476e-02
30,kirc_supervised__fold_2_200_genes_filtered,1.916276e-02
23,brca_unsupervised__fold_2_100_genes_filtered,2.286884e-02


In [19]:
acc = df[df['Name'].str.contains('acc')]
acc

,Name,Pval
11,acc_unsupervised__fold_2_100_genes_filtered,5.763423e-07
9,acc_unsupervised__fold_1_100_genes_filtered,1.139464e-04
12,acc_unsupervised__fold_2_200_genes_filtered,1.579201e-04
4,acc_supervised__fold_1_200_genes_filtered,4.103446e-03
6,acc_supervised__fold_2_200_genes_filtered,2.927964e-02
10,acc_unsupervised__fold_1_200_genes_filtered,3.038435e-02
3,acc_supervised__fold_1_100_genes_filtered,6.822511e-02
8,acc_unsupervised__fold_0_200_genes_filtered,1.324437e-01
5,acc_supervised__fold_2_100_genes_filtered,2.671960e-01
7,acc_unsupervised__fold_0_100_genes_filtered,3.156717e-01


In [20]:
brca = df[df['Name'].str.contains('brca')]
brca

,Name,Pval
22,brca_unsupervised__fold_1_200_genes_filtered,0.016855
23,brca_unsupervised__fold_2_100_genes_filtered,0.022869
15,brca_supervised__fold_1_100_genes_filtered,0.040853
24,brca_unsupervised__fold_2_200_genes_filtered,0.101533
21,brca_unsupervised__fold_1_100_genes_filtered,0.254484
16,brca_supervised__fold_1_200_genes_filtered,0.356115
18,brca_supervised__fold_2_200_genes_filtered,0.458310
19,brca_unsupervised__fold_0_100_genes_filtered,0.568693
20,brca_unsupervised__fold_0_200_genes_filtered,0.612878
17,brca_supervised__fold_2_100_genes_filtered,0.644442


In [21]:
kirc = df[df['Name'].str.contains('kirc')]
kirc

,Name,Pval
28,kirc_supervised__fold_1_200_genes_filtered,0.000006
27,kirc_supervised__fold_1_100_genes_filtered,0.000174
26,kirc_supervised__fold_0_200_genes_filtered,0.003755
30,kirc_supervised__fold_2_200_genes_filtered,0.019163
29,kirc_supervised__fold_2_100_genes_filtered,0.044287
25,kirc_supervised__fold_0_100_genes_filtered,0.049209
33,kirc_unsupervised__fold_1_100_genes_filtered,0.049291
36,kirc_unsupervised__fold_2_200_genes_filtered,0.092202
35,kirc_unsupervised__fold_2_100_genes_filtered,0.345557
31,kirc_unsupervised__fold_0_100_genes_filtered,0.355092
